In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem, rdFMCS
from rdkit.Chem import rdMolAlign
from rdkit.Chem import rdMolTransforms

ligands_path = "/path/to/ligands/to/be/aligned.sdf"
ref_ligand_path = "/path/to/ref/ligand.sdf"
output_path = "/path/to/output/ligands/aligned.sdf"

# Load reference ligand
ref_supplier = Chem.SDMolSupplier(ref_ligand_path, removeHs=False)
reference_ligand = ref_supplier[0]

# Load ligand series
ligand_supplier = Chem.SDMolSupplier(ligands_path, removeHs=False)

ligands = []
for lig in ligand_supplier:
    
    if lig is not None:
        ligands.append(lig)
    else:
        print("Error reading molecule from file")
        
# Find MCS between reference ligand and each ligand in the series
mcs_result = rdFMCS.FindMCS([reference_ligand] + ligands)
mcs_smarts = mcs_result.smartsString
mcs_mol = Chem.MolFromSmarts(mcs_smarts)

In [7]:
# Check if the reference ligand has 3D coordinates
if reference_ligand.GetNumConformers() == 0:
    print("The reference ligand does not have 3D coordinates!")
    
# Align ligands
for ligand in ligands:
    
    print("Aligning ligand: ", ligand.GetProp("_Name"))
    
    # Generate 3D coordinates if not present
    if ligand.GetNumConformers() == 0:
        print("Generating 3D coordinates...")
        AllChem.EmbedMolecule(ligand)
        AllChem.UFFOptimizeMolecule(ligand)
    
    # Get the atom indices for MCS in both ligands
    ref_match = reference_ligand.GetSubstructMatch(mcs_mol)
    lig_match = ligand.GetSubstructMatch(mcs_mol)
    
    if ref_match and lig_match:
        # Align ligand to reference using MCS atom indices
        rdMolAlign.AlignMol(ligand, reference_ligand, atomMap=list(zip(lig_match, ref_match)))

# Save aligned ligands
w = Chem.SDWriter(output_path)
for ligand in ligands:
    w.write(ligand)
w.close()

Aligning ligand:  PI3K_compound4
Aligning ligand:  PI3K_compound5
Aligning ligand:  PI3K_compound7
Aligning ligand:  PI3K_compound9
Aligning ligand:  PI3K_compound11
Aligning ligand:  PI3K_compound13
Aligning ligand:  PI3K_compound14
Aligning ligand:  PI3K_compound18
Aligning ligand:  PI3K_compound22
Aligning ligand:  PI3K_compound26
Aligning ligand:  PI3K_compound28
Aligning ligand:  PI3K_compound30
Aligning ligand:  PI3K_compound31
